In [ ]:
# 데이터 처리
import pandas as pd
import numpy as np
import openpyxl
from bs4 import BeautifulSoup

# selenium
from selenium import webdriver
# selenium으로 무엇인가 입력하기 위한 import
from selenium.webdriver.common.keys import Keys
# 구글 로그인하기 위한 모듈
import undetected_chromedriver as uc

#matplotlib
import matplotlib.pyplot as plt

# 시간 명시
from tqdm import tqdm
import time

# 경고 메시지 무시
import warnings
warnings.filterwarnings("ignore")



In [ ]:
#로그인, 구독한 채널 리스트, 최근 영상 리스트 크롤링하는 과정. 한번 실행하는데 성공하면 이후에는 따로 실행하지 않아도 됨. 
##크롬 드라이버 실행. 60초동안 로그인하기. 구글 로그인은 일반적인 크롬 드라이브로 로그인 시 블락함.
driver = uc.Chrome()
driver.get('https://youtube.com')
time.sleep(60)

##구독자 리스트 데이터프레임 만들기.
##구독자 리스트를 저장하기 위해 구독자 페이지로 이동.
driver.get('https://www.youtube.com/feed/channels')

time.sleep(5)
endkey = 5
for i in range(endkey):
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(1)

#구독한 채널 Dataframe 생성.
youtube_subscribe_dataframe = pd.DataFrame(columns=['channel_name', 'subscriber_num', 'impression_num'])


#구독한 채널 정보를 크롤링.
channel_list = driver.find_elements_by_class_name("style-scope ytd-channel-name")
subscriber_num_list = driver.find_elements_by_id("subscribers")

#구독하지 않은 영상을 카운트하기 위한 Unsubscribed행 추가.
data = {'channel_name': ['Unsubscribed'], 'subscriber_num': ['Blank'], 'impression_num': [0]}
youtube_subscribe_dataframe = youtube_subscribe_dataframe.append(pd.DataFrame(data))

#Dataframe에 크롤링한 채널 정보 추가.
for i in subscriber_num_list:
    youtube_subscribe_dataframe.loc[len(youtube_subscribe_dataframe)] = [channel_list[subscriber_num_list.index(i)].text, i.text, 0]
    youtube_subscribe_dataframe.loc[subscriber_num_list.index(i)+1] = [channel_list[subscriber_num_list.index(i)].text, i.text, 0]

#데이터 프레임 저장.
youtube_subscribe_dataframe.to_csv('youtube_subscribe_dataframe.csv', encoding='utf-8-sig')
youtube_subscribe_dataframe.to_csv('youtube_subscribe_dataframe_bak.csv', encoding='utf-8-sig')
subscribe_list = [] 

##최근 시청 기록 데이터프레임 만들기.
##최근 시청 기록 리스트를 저장하기 위해 최근 시청 기록 페이지로 이동.
driver.get('https://www.youtube.com/feed/history')

time.sleep(5)
endkey = 3
for i in range(endkey):
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(1)

#최근 시청 기록 Dataframe 생성.
youtube_history_dataframe = pd.DataFrame(columns=['channel_name', 'impression_num'])

#시청하지 않은 채널을 카운트하기 위한 행 추가.
data = {'channel_name': ['Never_seen'], 'impression_num': [0]}
youtube_history_dataframe = youtube_history_dataframe.append(pd.DataFrame(data))

#최근 시청 기록 정보를 크롤링.
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
channel_list = soup.findAll("a", {"class":"yt-simple-endpoint style-scope yt-formatted-string"})

#Dataframe에 크롤링한 채널 정보 추가.
for i in channel_list:
    youtube_history_dataframe.loc[len(youtube_history_dataframe)] = [i.text, 0]

#중복 제거, 인덱스 초기화
youtube_history_dataframe = youtube_history_dataframe.drop_duplicates()
youtube_history_dataframe.reset_index(drop=True, inplace=True)

#데이터 프레임 저장.
youtube_history_dataframe.to_csv('youtube_history_dataframe.csv', encoding='utf-8-sig')
youtube_history_dataframe.to_csv('youtube_history_dataframe_bak.csv', encoding='utf-8-sig')
history_list = [] 

In [ ]:
#유튜브 메인 화면 크롤링 함수 파트. youtube_main_crawling. 영상을 많이 크롤링하기 위해서는 반복작업이 필요하므로 위의 작업과는 달리 함수로 작성해야함.
def youtube_main_crawling(pagedown, save_and_go):
    #유튜브 메인 화면 접속 후 크롤링. 입력받은 수 만큼 페이지 다운 후 크롤링한다. 
    driver.get('https://youtube.com')
    time.sleep(3)
    endkey = pagedown
    for i in range(endkey):
        driver.find_element_by_tag_name('body').send_keys(Keys.END)
        time.sleep(1)

    #메인 화면에 나타나는 영상 정보 Dataframe 생성.
    youtube_main_dataframe = pd.DataFrame(columns=['channel_name', 'video_title'])

    #메인 화면에 나타나는 영상의 채널명을 크롤링.
    channel_list = driver.find_elements_by_class_name("style-scope ytd-channel-name")
    video_title_list = driver.find_elements_by_id("video-title")

    #크롤링한 데이터를 데이터프레임에 추가.
    for i in channel_list:
        try:
            youtube_main_dataframe.loc[channel_list.index(i)] = [i.text, video_title_list[channel_list.index(i)].text]
        except:
            IndexError

    #video_title에 '믹스'나 '내 관련 재생목록'이 포함되어 있으면 삭제한다.
    youtube_main_dataframe = youtube_main_dataframe[youtube_main_dataframe['video_title'].str.contains('믹스') == False]
    youtube_main_dataframe = youtube_main_dataframe[youtube_main_dataframe['video_title'].str.contains('내 관련 재생목록') == False]
    

    # youtube_main_dataframe의 'channel_name'값이 youtube_subscribe_dataframe의 'channel_name'값과 같으면
    # youtube_subscribe_dataframe의 'impression_num'값을 1증가시킨다.
    # 같지 않으면 'Unscubscribed'의 'impression_num'값을 1증가시킨다.
    # 같은 방식으로 youtube_history_dataframe도 작업.
    # youtube_subscribe_dataframe, youtube_history_dataframe은 전역변수이므로 함수 안에서 사용할 수 있게 global 선언.
    global youtube_subscribe_dataframe, youtube_history_dataframe

    # 꺾은선 그래프 그리기용
    global subscribe_list 
    global history_list

    youtube_subscribe_dataframe = pd.read_csv('youtube_subscribe_dataframe.csv', encoding='utf-8-sig', index_col = 0)
    for i in youtube_main_dataframe['channel_name']:
        if i in youtube_subscribe_dataframe['channel_name'].values:
            youtube_subscribe_dataframe.loc[youtube_subscribe_dataframe['channel_name'] == i, 'impression_num'] += 1
            subscribe_list.append(1)
        else:
            youtube_subscribe_dataframe.loc[youtube_subscribe_dataframe['channel_name'] == 'Unsubscribed', 'impression_num'] += 1
            subscribe_list.append(0)

    #save_and_go는 500개 이상의 영상을 크롤링할때, 여러번 결과를 저장하기 위해서 사용.
    if save_and_go == 1:
        youtube_subscribe_dataframe.to_csv('youtube_subscribe_dataframe.csv', encoding='utf-8-sig')

    # youtube_subscribe_dataframe의 'impression_num'값이 0이면 삭제한다. 
    youtube_subscribe_dataframe = youtube_subscribe_dataframe[youtube_subscribe_dataframe['impression_num'] >= 1]


    youtube_history_dataframe = pd.read_csv('youtube_history_dataframe.csv', encoding='utf-8-sig',index_col = 0)
    for i in youtube_main_dataframe['channel_name']:
        if i in youtube_history_dataframe['channel_name'].values:
            youtube_history_dataframe.loc[youtube_history_dataframe['channel_name'] == i, 'impression_num'] += 1
            history_list.append(1)
        else:
            youtube_history_dataframe.loc[youtube_history_dataframe['channel_name'] == 'Never_seen', 'impression_num'] += 1
            history_list.append(0)
            
    #save_and_go는 500개 이상의 영상을 크롤링할때, 여러번 결과를 저장하기 위해서 사용.
    if save_and_go == 1:
        youtube_history_dataframe.to_csv('youtube_history_dataframe.csv', encoding='utf-8-sig')
    
    

In [ ]:
## 함수 파트. represent_youtube_algorithm_subscribed
def represent_youtube_algorithm_subscribed():
    global youtube_subscribe_dataframe
    #총 크롤링한 영상 수
    total_crawling_video_num = youtube_subscribe_dataframe['impression_num'].sum()
    #크롤링한 영상 중 구독하지 않은 채널의 영상
    total_crawling_unsubscribed_video_num = youtube_subscribe_dataframe['impression_num'].loc[youtube_subscribe_dataframe['channel_name'] == 'Unsubscribed'].values[0]
    #크롤링한 영상 중 구독한 채널의 영상
    total_crawling_subscribed_video_num = total_crawling_video_num - total_crawling_unsubscribed_video_num

    
    print("총 크롤링한 영상 수: ", total_crawling_video_num)
    print("영상 중 구독하지 않은 채널의 영상 수 : ", total_crawling_unsubscribed_video_num)
    print("영상 중 구독한 채널의 영상 수 : ", total_crawling_subscribed_video_num)
    
    
    
    fig =plt.figure(figsize=(5,5))
    fig.set_facecolor('white')
    plt.title('Subscribed/Unsubscribed Ratio')
    ratio = [total_crawling_subscribed_video_num/total_crawling_video_num, total_crawling_unsubscribed_video_num/total_crawling_video_num]
    labels = ['Subscribed', 'Unsubscribed']
    explode = [0.05, 0.05]
    colors = ['#ff9999', '#ffc000']

    plt.pie(ratio, labels=labels, autopct='%.1f%%', startangle=260, counterclock=False, explode=explode, shadow=True, colors=colors)
    plt.show()

    p = total_crawling_subscribed_video_num/total_crawling_video_num
    plt.title('Ratio of subscribed/unsubscribed videos')
    cum_average = np.cumsum(subscribe_list)/(np.arange(len(subscribe_list))+1)
    plt.plot(cum_average)
    plt.plot([0,len(subscribe_list)-1],[p,p],'r--');
    plt.show()
    


In [ ]:
## 함수 파트. represent_youtube_algorithm_history
def represent_youtube_algorithm_history():
    global youtube_history_dataframe
    #최근에 봤던 채널의 수 
    total_watch_channel_num = len(youtube_history_dataframe)-1
    #총 크롤링한 영상 수
    total_crawling_video_num = youtube_history_dataframe['impression_num'].sum()
    #이전에 보지 않았던 채널인데 메인화면에 뜸.
    not_watch_main = youtube_history_dataframe['impression_num'].loc[youtube_history_dataframe['channel_name'] == 'Never_seen'].values[0]
    #이전에 봤던 채널인데 메인화면에는 뜨지않음.
    watch_not_main= youtube_history_dataframe['impression_num'].loc[youtube_history_dataframe['impression_num'] == 0].count()
    #이전에 봤던 채널이며 메인화면에도 뜨는 채널. -1 하는 이유는 Never_seen 은 봤던 채널이 아니기 때문.
    watch_main = youtube_history_dataframe['impression_num'].loc[youtube_history_dataframe['impression_num'] > 0].count() -1
    #메인화면에 뜨는 채널의 수
    channel_on_main = not_watch_main+watch_main




    print("최근에 봤던 채널의 수 : ", total_watch_channel_num)
    print("이전에 봤던 채널인데 메인화면에는 뜨지 않은 채널 수 : ", watch_not_main)
    print("이전에 봤던 채널인데 메인화면에도 뜨는 채널 수 : ", watch_main)
    fig =plt.figure(figsize=(5,5))
    fig.set_facecolor('white')
    plt.title('Watch Channel Ratio')
    ratio = [watch_not_main/total_watch_channel_num, watch_main/total_watch_channel_num]
    labels = ['Watched Channel but not on Main', 'Watched Channel on Main']
    explode = [0.05, 0.05]
    colors = ['pink', 'yellow']
    plt.pie(ratio, labels=labels, autopct='%.1f%%', startangle=260, counterclock=False, explode=explode, shadow=True, colors=colors)
    plt.show()




    print("메인화면에 나타나는 채널의 수 : ", channel_on_main)
    print("이전에 보지 않았던 채널인데 메인화면에 뜨는 채널 수 : ", not_watch_main)
    print("이전에 봤던 채널인데 메인화면에도 뜨는 채널 수 : ", watch_main)
    fig =plt.figure(figsize=(5,5))
    fig.set_facecolor('white')
    plt.title('New Channel/Watched Channel Ratio')
    ratio = [not_watch_main/channel_on_main, watch_main/channel_on_main]
    labels = ['New Channel on Main', 'Watched Channel on Main']
    explode = [0.05, 0.05]
    colors = ['orange', 'lightcoral']
    plt.pie(ratio, labels=labels, autopct='%.1f%%', startangle=260, counterclock=False, explode=explode, shadow=True, colors=colors)
    plt.show()
    

    
    p = sum(history_list) / len(history_list)
    plt.title('Probability of showing a video from a previously viewed channel')
    cum_average = np.cumsum(history_list)/(np.arange(len(history_list))+1)
    plt.plot(cum_average)
    plt.plot([0,len(history_list)-1],[p,p],'r--');
    plt.show()


In [ ]:
#함수 실행 3번, 페이지 다운 3번.
try_count = 5
page_down_count = 20
#해당 작업 반복 시 이전의 결과를 저장할건가? 1이면 저장하고 0이면 저장하지 않음. try_count가 2 이상이면 1로 해야 총 결과가 나옴.
save_and_go = 1


#초기화.
history_list = [] 
subscribe_list = [] 
#백업 파일 불러오기.
backup = pd.read_csv('youtube_history_dataframe_bak.csv', encoding='utf-8-sig',index_col = 0)
backup.to_csv('youtube_history_dataframe.csv', encoding='utf-8-sig')
backup = pd.read_csv('youtube_subscribe_dataframe_bak.csv', encoding='utf-8-sig',index_col = 0)
backup.to_csv('youtube_subscribe_dataframe.csv', encoding='utf-8-sig')


In [ ]:
## 유튜브 메인화면 크롤링 시작. 
for i in range(try_count):
    print("크롤링 중 . . . ")
    youtube_main_crawling(page_down_count, save_and_go)
    print("크롤링 {}회 완료".format(i+1))

print("크롤링 결과")
represent_youtube_algorithm_subscribed()
represent_youtube_algorithm_history()

print("크롤링 완료.")